# Fáza 3 - Strojové učenie (v 12. týždni): 20 bodov
Pri dátovej analýze nemusí byť naším cieľom získať len znalosti obsiahnuté v aktuálnych dátach, ale aj natrénovať model, ktorý bude schopný robiť rozumné predikcie pre nové pozorovania pomocou techniky strojového učenia. 
## Jednoduchý klasifikátor na základe závislosti v dátach  (5b)
Naimplementujte OneR algorithm (iné mená: OneRule or 1R), ktorý je jednoduchý klasifikátor tzv. rozhodnutie na základe jedného atribútu. Môžete implementovať aj komplikovanejšie t.j. rozhodnutie na základe kombinácie atribútov.
Algoritmus by mal byť realizovaný na základe závislostí v dátach. Vyhodnoťte klasifikátora pomocou metrík accuracy, precision a recall.
## Trénovanie a vyhodnotenie klasifikátorov strojového učenia (5b)
Na trénovanie využite minimálne jeden stromový algoritmus strojového učenia v scikit-learn.
Vizualizujte natrénované pravidlá.
Vyhodnoťte natrénované modely pomocou metrík accuracy, precision a recall
Porovnajte ašpoň jeden natrénovaný klasifikátor v scikit-learn s jednoduchým klasifikátorom z prvého kroku.
## Optimalizácia alias hyperparameter tuning (5b)
Preskúmajte hyperparametre Vášho zvoleného klasifikačného algoritmu v druhom kroku a vyskúšajte ich rôzne nastavenie tak, aby ste minimalizovali overfitting (preučenie) a optimalizovali výsledok. 
Vysvetlite, čo jednotlivé hyperparametre robia. Pri nastavovaní hyperparametrov algoritmu využite krížovú validáciu (cross validation) na trénovacej množine.
## Vyhodnotenie vplyvu zvolenej stratégie riešenia na klasifikáciu (5b) 
Vyhodnotíte Vami zvolené stratégie riešenia projektu z hľadiska classification accuracy: 
Stratégie riešenia chýbajúcich hodnôt a outlierov;
Scaling resp. transformer či zlepší accuracy klasifikácie;
Výber atribútov a výber algoritmov strojového učenia;
Hyperparameter tuning resp. ensemble learning.
Ktoré spôsoby z hore-uvedených bodov sa ukázali ako učinné pre Váš dataset? Hodnotenie podložíte dôkazmi. 

Správa sa odovzdáva v poslednom týždni semestra. Dvojica svojmu cvičiacemu odprezentuje vykonanú fázu v Jupyter Notebooku podľa potreby na cvičení. V notebooku uveďte percentuálny podiel práce členov dvojice. Následne správu elektronicky odovzdá jeden člen z dvojice do systému AIS do štvrtka 15.12.2022 23:59.

Naimportovanie knižníc, ktoré budeme používať

In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import tree
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.impute import KNNImputer
  



Nazačiatok si naimportujeme dáta, ktoré si hneď nakopírujeme, aby sme neprepisovali originály

In [17]:
train = pd.read_csv('training_data.csv',sep="\t")
test = pd.read_csv('testing_data.csv',sep="\t")

# Vytvorenie kopie datasetu
train_copy = train.copy()
test_copy = test.copy()

Trieda OneRule obsahuje inicialalizáciu, ktorá nám vytvorí prazdné premenné a metriku, ktorú budeme ohodnocovať klasifikáciu. 
Táto funkcia berie tréningové údaje ako vstup a používa ich na učenie sa súboru pravidiel, ktoré možno použiť na predpovedanie nových údajov. Funkcia iterovať cez trénovacie údaje a používame ich na identifikáciu najdôležitejších pravidiel. Tieto pravidlá možno potom použiť na predpovedanie nových údajov.

 Funkcia predict v algoritme OneRule sa používa na predpovedanie nových údajov na základe pravidiel, ktoré sa model naučil počas tréningu. 
 Táto funkcia aplikuje pravidlá na vstupné dáta, ktoré do nej vložíme

Funkcia score v algoritme OneRule sa používa na vyhodnotenie výkonu modelu v danom súbore údajov. Funkcia najprv použije funkciu predpovedania na generovanie predpovedí pre vstupné údaje a potom porovná predpovedané hodnoty so skutočnými hodnotami na výpočet presnosti modelu. Skóre presnosti sa potom môže použiť na vyhodnotenie výkonnosti modelu a určenie toho, ako dobre je schopný predpovedať nové údaje.

In [18]:
class OneRule:
    def __init__(self, metric):
        self.metric = metric
        self.rule = None
        self.rule_class = None

    def fit(self, X, y):
        classes = np.unique(y)
        best_rule = None
        best_rule_class = None
        best_score = 0
        for c in classes:
            rule = X[y == c].mean()
            score = self.score(X, y, rule, c)
            if score > best_score:
                best_score = score
                best_rule = rule
                best_rule_class = c
        self.rule = best_rule
        self.rule_class = best_rule_class

    def predict(self, X):
        return np.array([self.rule_class] * len(X))

    def score(self, X, y, rule, rule_class):
        y_pred = np.array([rule_class] * len(X))
        print(y_pred)
        y_pred[X < rule] = 1 - rule_class
        if self.metric == 'accuracy':
            return accuracy_score(y, y_pred)
        elif self.metric == 'precision':
            return precision_score(y, y_pred)
        elif self.metric == 'recall':
            # use zero_division=0 to avoid warning
            return recall_score(y, y_pred, zero_division="0")
        else:
            raise ValueError('Unknown metric: ' + self.metric)


Pripravíme si vstupy pre OneRule algoritmus
Aplikujeme OneRule algoritmus na stĺpec s parametrom warning, takže ho môžeme vyradiť zo základu pre testovanie.
X1 a X2 predstavujú hodnoty stĺpce, v ktorých nám v predchádzajúcich fázach zobrazili najvyššiu koreláciu so stĺpcom warning. 

In [19]:
X1 = train_copy["CH4"].values
X2 = train_copy["NOx"].values
y = train_copy['warning']

Aplikujeme OneRule algoritmus na stĺpec CH4 a zobrazíme výsledky

In [20]:

one_rule = OneRule("accuracy")
one_rule.fit(X1, y)

y_pred = one_rule.predict(X1)

print('Column: ' + "CH4")
print('Rule: ' + str(one_rule.rule))
print('Rule class: ' + str(one_rule.rule_class))
print('Accuracy: ' + str(accuracy_score(y, y_pred)))
print('Precision: ' + str(precision_score(y, y_pred)))
print('Recall: ' + str(recall_score(y, y_pred)))

[0 0 0 ... 0 0 0]
[1 1 1 ... 1 1 1]
Column: CH4
Rule: 0.6380420618263979
Rule class: 0
Accuracy: 0.4030812324929972
Precision: 0.0
Recall: 0.0


c:\Users\mattp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Aplikujeme OneRule algoritmus na stĺpec NOx a zobrazíme výsledky

In [21]:
one_rule = OneRule("accuracy")
one_rule.fit(X2, y)

y_pred = one_rule.predict(X2)

print('Column: ' + "NOx")
print('Rule: ' + str(one_rule.rule))
print('Rule class: ' + str(one_rule.rule_class))
print('Accuracy: ' + str(accuracy_score(y, y_pred)))
print('Precision: ' + str(precision_score(y, y_pred)))
print('Recall: ' + str(recall_score(y, y_pred)))

[0 0 0 ... 0 0 0]
[1 1 1 ... 1 1 1]
Column: NOx
Rule: 0.7173573186124478
Rule class: 0
Accuracy: 0.4030812324929972
Precision: 0.0
Recall: 0.0


c:\Users\mattp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Na rovnaký algoritmus použijeme DecisionTreeClassifier() z knižnice sklearn za účelom porovnania dát

Najprv si pripravíme dáta na vstup, rovnako v prípade vyššie

In [31]:
X1 = train_copy["CH4"]
X2 = train_copy["NOx"]
y = train_copy['warning']

Vkladáme dáta do DecisionTreeClassifier a spustíme funkciu predikácie

DecisionTreeClassifier je typ algoritmu učenia, ktorý možno použiť na klasifikačné úlohy podobne ako OneRule algoritmus.

Funkciu reshape používame, iba aby sme na vstupe do funkcie fit mohli použiť 1D zoznam.

In [45]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X1.values.reshape(-1,1), y.reshape(-1,1))
y1_pred = clf.predict(X1.values.reshape(-1,1))

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X2.values.reshape(-1,1), y.reshape(-1,1))
y2_pred = clf.predict(X2.values.reshape(-1,1))



Nakoniec si zobrazíme výsledky

In [47]:
print('Column: ' + "CH4")
print('Accuracy: ' + str(accuracy_score(y, y1_pred)))
print('Precision: ' + str(precision_score(y, y1_pred)))
print('Recall: ' + str(recall_score(y, y1_pred)))

print('Column: ' + "NOx")
print('Accuracy: ' + str(accuracy_score(y, y2_pred)))
print('Precision: ' + str(precision_score(y, y2_pred)))
print('Recall: ' + str(recall_score(y, y2_pred)))


Column: CH4
Accuracy: 0.9703081232492997
Precision: 0.9582139085834968
Recall: 0.9935867354919443
Column: NOx
Accuracy: 0.9724556489262371
Precision: 0.9954501137471563
Recall: 0.9582355701548568


Identitický postup použijeme na otestovanie ďalšieho stromového modelu - RandomForestClassifier.

In [50]:
X1 = train_copy["CH4"]
X2 = train_copy["NOx"]
y = train_copy['warning']

clf = RandomForestClassifier()
clf = clf.fit(X1.values.reshape(-1,1), y.values.reshape(-1,1))
y1_pred = clf.predict(X1.values.reshape(-1,1))

clf = RandomForestClassifier()
clf = clf.fit(X2.values.reshape(-1,1), y.values.reshape(-1,1))
y2_pred = clf.predict(X2.values.reshape(-1,1))

print('Column: ' + "CH4")
print('Accuracy: ' + str(accuracy_score(y, y1_pred)))
print('Precision: ' + str(precision_score(y, y1_pred)))
print('Recall: ' + str(recall_score(y, y1_pred)))

print('Column: ' + "NOx")
print('Accuracy: ' + str(accuracy_score(y, y2_pred)))
print('Precision: ' + str(precision_score(y, y2_pred)))
print('Recall: ' + str(recall_score(y, y2_pred)))


C:\Users\mattp\AppData\Local\Temp\ipykernel_16932\3065635485.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf = clf.fit(X1.values.reshape(-1,1), y.values.reshape(-1,1))
C:\Users\mattp\AppData\Local\Temp\ipykernel_16932\3065635485.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf = clf.fit(X2.values.reshape(-1,1), y.values.reshape(-1,1))


Column: CH4
Accuracy: 0.9700280112044818
Precision: 0.9553089382123575
Recall: 0.9964023150320663
Column: NOx
Accuracy: 0.9722689075630252
Precision: 0.9912959381044487
Recall: 0.9619896762083529


Porovnanie výsledkov